In [1]:
### Import libraries
import openai
import random
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from helper import *
from stories import *
from pipelines import *
import psycopg2

In [2]:
now = datetime.now()
date_time = now.strftime("%Y%m%d")

## OpenAI

In [3]:
set_api_key()

## Connect to Database

In [4]:
conn = get_connection()

## Load Projects

In [5]:
projects = get_useful_projects()

## Functions

In [6]:
def save_in_database(conn, prompt1, prompt2, encoding, story, story_1, story_2, version, file):
    cursor = conn.cursor()
    command = "INSERT INTO ne_linebyline (prompt1, prompt2, encoding, story, story_1, story_2, version, file) VALUES ('%(prompt1)s', '%(prompt2)s', '%(encoding)s', '%(story)s', '%(story_1)s', '%(story_2)s', '%(version)s', '%(file)s');" % {"prompt1": prompt1, "prompt2": prompt2, "encoding": encoding, "story": story, "story_1": story_1, "story_2":story_2, "version":version, "file":file}
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=}")
        conn.commit()
        cursor.close()
        pass

In [7]:
def get_2_random_projects(projects,project):
    filtered_projects = [element for element in projects if element != project]
    project_1, project_2 = random.sample(filtered_projects,2)
    return project_1, project_2

In [8]:
def quality_check(name_file, lines_file, previous_messages, instance, deep):
    if deep==0:
        return lines_file[-1]
    with open(name_file,"w") as tmp_file:
        for new_line in lines_file:
            tmp_file.write(new_line)
    models, errors, symbols, mistakes = asp_try_5(name_file,instance)
    #print(f"Errors {errors}. Messages {mistakes}.")
    new_answer = lines_file[-1]
    if len(mistakes) > 0:
        print("Mistake! Last Message : " + str(previous_messages[-1]))
        answer = get_completion_line('''The last rule that you wrote is wrong. The error message is: 
                                    ###" + str(mistakes) + "###.
                                    Write that last rule again, correcting it, without apologizing or writing any explanation.'''
                                    , previous_messages = previous_messages)
        lines_file[-1] = answer
        print("Deep: "+ str(deep) + ". New answer: " +answer)
        new_answer = quality_check(name_file,lines_file,previous_messages, instance, deep-1)
    return new_answer

## Main : From Script, Line4Line, ICL, TryAgain

In [ ]:
for project in projects:
    story = project["story"]
    encoding = project["lines"]
    instance = instances_dict[story]
    problem = project["problem"]
    representation = project["Representation in ASP"]
    content = f'''Problem 3: 
                    {problem}.
                  Representation 3:
                    {representation}'''
    lines = encoding.splitlines()
    new_file = []
    project_1, project_2 = get_2_random_projects(projects,project)
    story_1 = project_1["story"]
    story_2 = project_2["story"]
    content1 = f'''Problem 1: {project_1["problem"]}
                    Representation 1: {project_1["Representation in ASP"]}
                    Encoding 1: {project_1["lines"]}  '''
    content2 = f'''Problem 2: {project_2["problem"]}
                    Representation 2: {project_2["Representation in ASP"]}
                    Encoding 2: {project_2["lines"]}  '''
    messages = [{"role":"user","content":content1}]
    messages.append({"role":"user","content":content2})
    messages.append({"role":"user","content":content})
    new_file_name = story + "_from_" + story_1 + "_" + story_2 + "_v18_SLIT_" + date_time + ".lp"
    bad_file = False
    for line in lines:
        if line == '\n':
            continue
        if len(line.strip()) > 0 and line.strip()[0] == '%':
            print("---------------------------------------------------")
            new_file.append("\n" + line)
            messages.append({"role":"user","content":line})
            print(line)
            answer = get_completion_representation(line,previous_messages=messages)
            print(answer)
            new_file.append("\n" + answer)
            messages.append({"role":"assistant","content":answer})
            if not bad_file:
                new_answer = quality_check(new_file_name,new_file,messages,instance,deep=3)
                new_file[-1] = new_answer
                with open(new_file_name,"w") as tmp_file:
                    for new_line in new_file:
                        tmp_file.write(new_line)
                models, errors, symbols, mistakes = asp_try_5(new_file_name,instance)
                if len(mistakes) > 0:
                    bad_file = True
    os.remove(new_file_name)
    prompt = '''Given the description of a problem, predicates in Answer Set Programming, and a commented line, 
                your task is to translate this last commented line to a rule in Answer Set Programming, using the given predicates or past rules. 
                Just write one or a few predicates or rules. Do not try to solve the whole problem at once.'''
    encoding = ' '.join(new_file)
    save_in_database(conn,prompt,'',encoding.replace("'",""),story,story_1, story_2, 18, new_file_name)  

---------------------------------------------------
% Four atoms with predicate "plus" of 2 variables with possible values 0 and 1.
plus(0,1).
plus(1,0).
plus(0,-1).
plus(-1,0).
---------------------------------------------------
% The four possible directions, predicate "dir" with 2 variables, of value 0, 1 or -1. dir(0,0) is not an option.
dir(0,1). 
dir(0,-1). 
dir(1,0). 
dir(-1,0).
---------------------------------------------------
% Predicate "black" of 2 variables is generated as the sum of the sum of the position (X,Y) and (A,B), for plus (A,B), if the cell(X+A,Y+B) exists, given hint(X,Y,N). There should be exactly "N" "black" predicates generated.
N { black(X+A,Y+B) : plus(A,B), cell(X+A,Y+B) } N :- hint(X,Y,N).
---------------------------------------------------
% Predicate "path" has 4 variables, indicating a path between two cells of predicate "white" of positions (X,Y) and (P,Q) if the sum of the absolute values of the differences of P and X, and Q and Y, is 1. Absolute v

In [ ]:
new_file

In [ ]:
!clingo test_light.lp projects/lights/instances/test01.lp

In [2]:
!clingo 0 test_light.lp projects/lights/instances/test01.lp

clingo version 5.6.2
Reading from test_light.lp ...
Solving...
Answer: 1
light(4,1) light(4,3) light(3,4)
Answer: 2
light(1,1) light(4,1) light(4,3) light(3,4)
Answer: 3
light(4,1) light(4,3) light(2,4) light(3,4)
Answer: 4
light(1,1) light(4,1) light(4,3) light(2,4) light(3,4)
Answer: 5
light(4,1) light(2,2) light(4,3) light(3,4)
Answer: 6
light(4,1) light(2,2) light(4,3) light(2,4) light(3,4)
Answer: 7
light(1,1) light(4,1) light(2,2) light(4,3) light(3,4)
Answer: 8
light(1,1) light(4,1) light(2,2) light(4,3) light(2,4) light(3,4)
Answer: 9
light(4,1) light(4,2) light(4,3) light(3,4)
Answer: 10
light(1,1) light(4,1) light(4,2) light(4,3) light(3,4)
Answer: 11
light(4,1) light(2,2) light(4,2) light(4,3) light(3,4)
Answer: 12
light(1,1) light(4,1) light(2,2) light(4,2) light(4,3) light(3,4)
Answer: 13
light(4,1) light(4,2) light(4,3) light(2,4) light(3,4)
Answer: 14
light(4,1) light(2,2) light(4,2) light(4,3) light(2,4) light(3,4)
Answer: 15
light(1,1) light(4,1) light(4,2) light(4,3) 